In [1]:
#for device number check
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Aug 11 09:35 /dev/video0


In [2]:
from jetcam.usb_camera import USBCamera

# for USB Camera
camera = USBCamera(width=224, height=224, capture_device=0) 

camera.running = True
print("camera ready for use")

camera ready for use


In [3]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'hands'

CATEGORIES = ['describe', 'think', 'worry', 'excited']

DATASETS = ['A', 'B']


TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('../data/classification/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

hands task with ['describe', 'think', 'worry', 'excited'] categories defined


In [4]:
# Set up the data directory location
DATA_DIR = '/nvdli-nano/data/classification/'
!mkdir -p {DATA_DIR}

In [5]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera 
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image(layout=ipywidgets.Layout(width="90%"))
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets for data collection
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset', layout=ipywidgets.Layout(width="70%"))
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category', layout=ipywidgets.Layout(width="70%"))
count_widget = ipywidgets.IntText(description='count', layout=ipywidgets.Layout(width="70%"))
save_widget = ipywidgets.Button(description='add', layout=ipywidgets.Layout(width="70%"))

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)


print("widgets for data collection are created")

widgets for data collection are created


In [6]:
import torch
import torchvision


device = torch.device('cuda')

# MOBILENETV2
#model = torchvision.models.mobilenet_v2(pretrained=True)
#model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 34
#model = torchvision.models.resnet34(pretrained=True)
#model.fc = torch.nn.Linear(512, len(dataset.categories))
    
model = model.to(device)

model_save_button = ipywidgets.Button(description='save model', layout=ipywidgets.Layout(width="70%"))
model_load_button = ipywidgets.Button(description='load model', layout=ipywidgets.Layout(width="70%"))
model_path_widget = ipywidgets.Text(description='model path', value='/nvdli-nano/data/classification/my_model.pth', layout=ipywidgets.Layout(width="70%"))

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

print(model)

print("model is configured")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop', layout=ipywidgets.Layout(width="25%"))
prediction_widget = ipywidgets.Text(description='prediction', layout=ipywidgets.Layout(width="90%"))


def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        prediction_widget.value = dataset.categories[category_index]
        
            
def start_live(change):
    if change['new'] == 'live':
        #execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')


print("live_execution part is created")

live_execution part is created


In [8]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())


epochs_widget = ipywidgets.IntText(description='epochs', value=1, layout=ipywidgets.Layout(width="70%"))
eval_button = ipywidgets.Button(description='evaluate', layout=ipywidgets.Layout(width="70%"))
train_button = ipywidgets.Button(description='train', layout=ipywidgets.Layout(width="70%"))
loss_widget = ipywidgets.FloatText(description='loss', layout=ipywidgets.Layout(width="70%"))
accuracy_widget = ipywidgets.FloatText(description='accuracy', layout=ipywidgets.Layout(width="70%"))
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress', layout=ipywidgets.Layout(width="70%"))

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    

# display(train_eval_widget)
print("trainer is configured and trainer evaluator has beed created")

trainer is configured and trainer evaluator has beed created


In [9]:
# Combine all the widgets into one display

combined_widget = ipywidgets.AppLayout(header=None, 
          left_sidebar=ipywidgets.VBox([dataset_widget, category_widget, count_widget, save_widget, epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    ipywidgets.VBox([train_button, eval_button])
]),
          center= ipywidgets.VBox([camera_widget]),
          right_sidebar=ipywidgets.VBox([prediction_widget,state_widget, model_path_widget,
    ipywidgets.VBox([model_load_button, model_save_button])
]),
          footer=None)

all_widget = ipywidgets.VBox([
    ipywidgets.HBox([combined_widget])
])

display(all_widget)